In [1]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd

project = 'analytics-trafic-idfm' # Project ID inserted based on the query results selected to explore
location = 'EU' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=analytics-trafic-idfm:EU:bquxjob_4a9f0a49_19566b9091d)
back to BigQuery to edit the query within the BigQuery user interface.

In [2]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_4a9f0a49_19566b9091d') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT
  centroid_id AS cluster_id,
  ROUND(AVG(CASE WHEN feature = 'energy_100g' THEN numerical_value END), 2) AS avg_energy,
  ROUND(AVG(CASE WHEN feature = 'fat_100g' THEN numerical_value END), 2) AS avg_fat,
  ROUND(AVG(CASE WHEN feature = 'saturated_fat_100g' THEN numerical_value END), 2) AS avg_saturated_fat,
  ROUND(AVG(CASE WHEN feature = 'carbohydrates_100g' THEN numerical_value END), 2) AS avg_carbohydrates,
  ROUND(AVG(CASE WHEN feature = 'sugars_100g' THEN numerical_value END), 2) AS avg_sugars,
  ROUND(AVG(CASE WHEN feature = 'fiber_100g' THEN numerical_value END), 2) AS avg_fiber,
  ROUND(AVG(CASE WHEN feature = 'proteins_100g' THEN numerical_value END), 2) AS avg_proteins,
  ROUND(AVG(CASE WHEN feature = 'salt_100g' THEN numerical_value END), 2) AS avg_salt,
  ROUND(AVG(CASE WHEN feature = 'additives_count' THEN numerical_value END), 2) AS avg_additives_count
FROM 
  ML.CENTROIDS(MODEL `analytics-trafic-idfm.food_consumption_trends.kmeans_nutrition_model`)
GROUP BY clust

# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API. The read_gbq_table [method](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.pandas#bigframes_pandas_read_gbq_function) turns a BigQuery table into the BigFrames DataFrame.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [3]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_4a9f0a49_19566b9091d') # Job ID inserted based on the query results selected to explore
destination = job.destination

# Load data from a BigQuery table using BigFrames DataFrames:
bq_df = bpd.read_gbq(f"{destination.project}.{destination.dataset_id}.{destination.table_id}")
bq_df

,cluster_id,avg_energy,avg_fat,avg_saturated_fat,avg_carbohydrates,avg_sugars,avg_fiber,avg_proteins,avg_salt,avg_additives_count
0,4,0.01,11.81,5.35,35.48,27.23,1.9,5.59,0.71,11.93
1,2,0.0,4.89,21.92,16.61,11.52,19.57,50.34,0.3,1.19
2,5,0.01,53.36,26.39,7.43,9.75,4.43,15.75,1.19,1.8
3,3,3.8,2.64,1.5,5.9,3.57,1.04,4.17,0.52,1.72
4,1,0.0,10.81,4.47,61.55,28.66,3.26,6.28,1.03,2.34


## Show descriptive statistics using describe()
Use the ```DataFrame.describe()```
[method](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.dataframe.DataFrame#bigframes_dataframe_DataFrame_describe)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [4]:
# Generate descriptive statistics.
bq_df.describe()

,cluster_id,avg_energy,avg_fat,avg_saturated_fat,avg_carbohydrates,avg_sugars,avg_fiber,avg_proteins,avg_salt,avg_additives_count
count,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
mean,3.0,0.764,16.702,11.926,25.394,16.146,6.04,16.426,0.75,3.796
std,1.581139,1.697183,20.854972,11.364688,23.392647,11.179442,7.673347,19.501193,0.363662,4.565274
min,1.0,0.0,2.64,1.5,5.9,3.57,1.04,4.17,0.3,1.19
25%,2.0,0.0,4.89,4.47,7.43,9.75,1.9,5.59,0.52,1.72
50%,3.0,0.01,10.81,5.35,16.61,11.52,3.26,6.28,0.71,1.8
75%,4.0,0.01,11.81,21.92,35.48,27.23,4.43,15.75,1.03,2.34
max,5.0,3.8,53.36,26.39,61.55,28.66,19.57,50.34,1.19,11.93


In [5]:
# Convert BigQuery DataFrame to pandas DataFrame.
pandas_df = bq_df.to_pandas()

In [7]:
pandas_df

,cluster_id,avg_energy,avg_fat,avg_saturated_fat,avg_carbohydrates,avg_sugars,avg_fiber,avg_proteins,avg_salt,avg_additives_count
0,4,0.01,11.81,5.35,35.48,27.23,1.9,5.59,0.71,11.93
1,2,0.0,4.89,21.92,16.61,11.52,19.57,50.34,0.3,1.19
2,5,0.01,53.36,26.39,7.43,9.75,4.43,15.75,1.19,1.8
3,3,3.8,2.64,1.5,5.9,3.57,1.04,4.17,0.52,1.72
4,1,0.0,10.81,4.47,61.55,28.66,3.26,6.28,1.03,2.34


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['cluster_id'].plot(kind='hist', bins=20, title='cluster_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['avg_energy'].plot(kind='hist', bins=20, title='avg_energy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['avg_fat'].plot(kind='hist', bins=20, title='avg_fat')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='avg_fat', y='avg_saturated_fat', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['cluster_id'].plot(kind='line', figsize=(8, 4), title='cluster_id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['avg_energy'].plot(kind='line', figsize=(8, 4), title='avg_energy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['avg_fat'].plot(kind='line', figsize=(8, 4), title='avg_fat')
plt.gca().spines[['top', 'right']].set_visible(False)